In [17]:
#Import Libraries 
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
from datetime import timedelta

load_dotenv()


True

In [18]:
#Define today and month
today = '2024-05-29'
month = '2024-04'



## 1. Load the pending result file to python

In [19]:
# Load all pending dfs in a folder
# dfs = load_all_pending_dfs(Path("data",today))

In [20]:
def load_single_pending_df(path_:str) -> pd.DataFrame:
    """Load pending result DataFrame"""
    
    df=pd.read_excel(path_,skiprows=1, engine='xlrd')
    return df

In [21]:
# coba_load = load_single_pending_df(Path("data",today,"Pending Results.xls"))
# coba_load.head()

In [22]:
def load_all_pending_dfs(path_:str) -> list[pd.DataFrame]:
    """Load all pending result dataframe, return them as a list of dataframe"""
    # Load dataframes for each file
    offline_classroom = load_single_pending_df(Path(path_, "Pending Results.xls"))
    offline_other = load_single_pending_df(Path(path_, "Pending Results(1).xls"))
    online_classroom = load_single_pending_df(Path(path_, "Pending Results(2).xls"))
    online_other = load_single_pending_df(Path(path_, "Pending Results(3).xls"))
    
    # Store dataframes in a list
    return [offline_classroom, offline_other, online_classroom, online_other]

In [23]:
# dfs = load_all_pending_dfs(Path("data",today))
# dfs[0].head(20)

## 2. Clean Data

In [24]:
#Clean data
# df_clean = clean_pending_df(dfs,today,month)

In [25]:
# Funtion clean_trainer_name
# Note: in Series, we have to specified what we are work on. that why we must use str

def clean_trainer_name(df:pd.DataFrame, teacher_col:str) -> pd.Series:
    """clean teacher's name"""
    
    teachers = (
        df[teacher_col]
        .str.title() #capital in the beginning of each word
        .str.replace("\(.+\)", "", regex=True) #replace data inside () with blank. ex: (Math)
        .str.replace("\s+", " ", regex=True) #Replace extra whitespace in the middle of words with single space
        .str.strip() #Remove extra whitespace in the beginning and end of words
        .replace(
            {
                "Azhar Rahul": "Azhar Rahul Finaya",
                "Handayani Risma": "Handayani Khaerunisyah Risma",
                "Kartikasari Prettya": "Kartikasari Prettya Nur",
                "Ramadhan Ira Ragil": "Ramadhani Ira",
                "S Allan": "Santiago Allan",
                "Gandhama Jesita": "Ghandama Jesita",
                "Istiqomah Diah": "Toluhula Diah Istiqomah",
                "Putri Tiara": "Setiawan Tiara Putri",
                "Hamsah Ratnasari Handayani": "Hamsah Handayani Ratnasari"
                
            }
        )
        
    )
    
    return teachers

#Call function pakai data frame coba_load untuk lihat perubahan data
# coba_coba= clean_trainer_name(coba_load, "Teacher")
# coba_coba.head()


In [26]:
#Function load_trainer_df
def load_trainer_df(month: str) -> pd.DataFrame:
    """Load file trainer_working_days"""
    
    path= os.getenv("path_trainer_data") #Path untuk lokasi file trainer_working_days ini ada di file .env
    df_trainer= pd.read_excel(path, sheet_name=month) #nama sheet di file trainer_working_days itu sama dengan month yg di define di awal
    
    
    return df_trainer

#Call out the function
# load_trainer_df(month).head()

In [27]:
# Clean data using chaining method
def clean_pending_df(dfs: list, date_exported: str, month: str) -> pd.DataFrame:
    """Clean pending result dataframe to obtain list of pending results per session"""
   
    df_clean =(
        pd.concat(dfs) # gabung 4 file pending result jadi satu
        .drop( #hapus kolom
            columns=["Unnamed: 6","Level / Unit", "First Name", "Last Name", "Code", "Service Type"]
        )
        .drop_duplicates( #hapus duplicated row untuk tahu jadwal kelas dari setiap teacher
            subset=["Teacher", "Class Type", "Date", "Start Time"], keep="first"
        )
        .loc[lambda df_: df_["Class Type"] != "Staff Appointment"]
        .rename(columns = lambda c: c.lower().replace("_",""))
        .dropna(subset=["teacher"])
        .dropna(how="all",axis=0) #Delete empty rows
        .dropna(how="all", axis=1) #Delete empty column
        .assign( #modify/overwritten the column name "teacher"
            teacher=lambda df_: clean_trainer_name(
                df_, "teacher" 
                # lambda function that takes df_ as input and applies the function clean_trainer_name() to the "teacher" column. 
            ),
            date= lambda df_: pd.to_datetime(df_["date"]), # Modify/overwritten the column 'date' to datetime format
            date_exported= pd.to_datetime(date_exported) #Create New Column "date_exported"
        )
        # merge with et data to get area and position
        .merge(
            right=load_trainer_df(month), #Merge dfs (left table) with the result of load_trainer_df function (right table)
            left_on= "teacher", # key join dari column di dfs
            right_on="coco_teacher_name", #Key join dari hasil function load_trainer_df
            how="left" #pakai Left join untuk merge data
        )
        #Drop unused columns
        .drop(
            columns=[
                "coco_teacher_name",
                "erwin_teacher_name",
                "teacher_working_days",
                "teacher_note_1",
                "teacher_note_2",
                "center name"
            ]
        )
        # Sort columns
        # tanda : artinya kita akan akses semua baris dari column yang disebutkan
        .loc[
            :,
            [
                "teacher",
                "date",
                "start time",
                "class type",
                "teacher_position",
                "teacher_center",
                "teacher_area",
                "date_exported",
            ],
        ]
        # sorts the DataFrame based on multiple columns in ascending order. 
        .sort_values(["teacher_area", "teacher_center", "teacher", "date"])
        .reset_index(drop=True) # resets the index of the DataFrame after sorting and drops the old index.
        # Exclude some ET name that already resigned
        .loc[lambda df_: ~df_["teacher"].isin(["Jane Quinn Madeline",
            "Priscilla Yokhebed",
            "Rifani Aurora Nurhidayah",
            "Nasarah Nadya",
             "Louei Frentzen Caesar",
            "Bushey James Michael",
            "Mowatt Peter Denis",
             "Azhar Rahul Finaya",
            "Kartikasari Prettya Nur",
             "Laurendeau Derek",
             "Rozak Abdul Rahman"])]
        #Keep only the data from the last 365 days
        .loc[lambda df_: df_["date"] >= (pd.to_datetime(date_exported) - timedelta(days=365))]

    )

    return df_clean

# df_clean = clean_pending_df(dfs,today,month)
# df_clean.tail()



In [28]:
#cek how may duplicated rows
# print(df_clean.duplicated().sum())

In [29]:
# df_clean = clean_pending_df(dfs,today,month)
# df_clean.head()


## 3. Transform Data (Get summary of pending result)

In [30]:
# # count pending result per month / create summary
# df_pending = count_pending_result(df_clean, today)

In [31]:
#function count_pending_result

def count_pending_result(df: pd.DataFrame, today: str) -> pd.DataFrame:
    """Get summary of pending result in the last 365 days,
    grouped by area and teacher, pivoted per month.
    """
    
    return(
        df
         # This filters the DataFrame to include only the data from the past 365 days.
        .loc[lambda df_: (pd.to_datetime(today) - df_["date"]).dt.days <= 365]
        #Group the data and "date" column grouped by monthly frequency (1M)
        .groupby(["teacher_area", "teacher", pd.Grouper(key="date", freq="1M")])
        # counting how many times each unique teacher's name shows up in the "teacher" column within each group defined by "teacher_area" and the month
        # save the output in new column called num_session_with_pending_res
        .agg(num_session_with_pending_res=("teacher","count"))
        #Karena teacher area & teacher column jadi index, kita reset
        .reset_index()
        #Pivot the table where teacher area & teacher jadi index, date jadi column dan num_session_with_pending_res jadi value
        .pivot(index=["teacher_area", "teacher"], columns="date")
        # # Replace all NaN elements with 0s.
        .fillna(0)
        # note: do not display trainer if the last 3 months pending results is 0
        # .loc[lambda df_: df_.iloc[:, -3:].sum(axis=1) != 0]
        # dropping the first (top) level of the column index, which contains the label "num_session_with_pending_res"
        # axis =1 means we are working on the column level. drop the top level from the columns of the DataFrame.
        .droplevel(0, axis=1)
        #Sort the column based on the newest date to the oldest date
        .sort_index(axis="columns", ascending=False)
        # renames each column label (which represents dates) into a more readable format
        .rename(columns= lambda c: c.strftime("%b %Y"))
        
        
        # #Rename index level (teacher_area and teacher are become index here). rename_axis itu secara default 0 axisnya (change index level)
        .rename_axis(["Teacher Area", "Teacher"])
        #Rename the column "date" with blank. jadi cuma hapus tulisan "date" aja
        .rename_axis([""], axis=1)
    )
    


In [32]:
# # count pending result per month / create summary
# df_pending = count_pending_result(df_clean, today)
# df_pending.head(15)

## 3. Load file file in the folder

In [33]:
# Load all pending dfs in a folder
dfs= load_all_pending_dfs(Path("data",today))

#Clean data
df_clean= clean_pending_df(dfs,today,month)

#Count pending result per month/create summary
df_pending= count_pending_result(df_clean, today)

WARNING *** file size (180835) not 512 + multiple of sector size (512)
WARNING *** file size (1106531) not 512 + multiple of sector size (512)
WARNING *** file size (6819) not 512 + multiple of sector size (512)
WARNING *** file size (213603) not 512 + multiple of sector size (512)


In [34]:
# Function to check that all the teachers exist in coco trainer data

def test_all_teacher_exist_in_coco_trainer_data(df_clean):
    # Check if the teacher area is null, then show the teacher name
    unmapped= df_clean.loc[df_clean["teacher_area"].isna(), "teacher"].unique()
    # Check if the result is zero or not. if not zero, it will print the list of teachers that is not list down in coco trainer data
    assert(
        unmapped.shape[0] == 0   
    ), f"some teacher are not listed in coco_trainer_data: {unmapped}"
    
test_all_teacher_exist_in_coco_trainer_data(df_clean)
    

In [35]:
print(df_clean.columns)

Index(['teacher', 'date', 'start time', 'class type', 'teacher_position',
       'teacher_center', 'teacher_area', 'date_exported'],
      dtype='object')


In [36]:
# create data per area
df_clean= df_clean.rename(columns= lambda c: c.replace("_"," ").title()) #  replace underscores with spaces and title case the column names (capitalize the first letter of each word).

# These lines create separate DataFrames for each teacher area.
df_jkt1= df_clean.loc[df_clean["Teacher Area"] == "JKT 1"]
df_jkt2= df_clean.loc[df_clean["Teacher Area"] == "JKT 2"]
df_jkt3= df_clean.loc[df_clean["Teacher Area"] == "JKT 3"]
df_sby= df_clean.loc[df_clean["Teacher Area"] == "SBY"]
df_bdg= df_clean.loc[df_clean["Teacher Area"] == "BDG"]
df_onl= df_clean.loc[df_clean["Teacher Area"].isin(["Online", "Shared Account", "Ooolab"])]
df_oth= df_clean.loc[df_clean["Teacher Area"].isin(["Other","HO"])]


We need to check that the number of row that breakdown by the area is the same as the total row in df_clean

In [37]:

#This line asserts that the total number of rows in the original DataFrame df_clean is equal to the sum of rows in all the separate DataFrames created for each area.
# It ensures that no rows are missed during the filtering process. If the total rows don't match, it will raise an error.
# assert that no rows are missed
assert len(df_clean) == sum(
    [len(df) for df in [df_jkt1, df_jkt2, df_jkt3, df_sby, df_bdg, df_onl, df_oth]]
)


In [38]:
# Check the total row of df_clean and the other df
clean_row= len(df_clean)
output_row= len(df_jkt1)+len(df_jkt2)+len(df_jkt3)+len(df_bdg)+len(df_sby)+len(df_onl)+len(df_oth)

print(clean_row, output_row)

397 397


In [39]:
# List the data which is left out from the df_clean (if any)

import pandas as pd

# Assuming df_clean, df_jkt1, df_jkt2, df_jkt3, df_bdg, df_sby, df_onl, df_oth are your dataframes

# Concatenate all other dataframes into one
df_all_others = pd.concat([df_jkt1, df_jkt2, df_jkt3, df_bdg, df_sby, df_onl, df_oth])

# Find rows in df_clean that are not in any other dataframe
filtered_df = df_clean[~df_clean['Teacher'].isin(df_all_others['Teacher'])]

# Now filtered_df contains the rows from df_clean that are not in any other dataframe

filtered_df


,Teacher,Date,Start Time,Class Type,Teacher Position,Teacher Center,Teacher Area,Date Exported


Let's save the output 

In [41]:
# save df
# write each dataframe to different worksheet

filename= "output.xlsx"
filepath= os.path.join("data", today, filename)
writer= pd.ExcelWriter(filepath, engine="xlsxwriter")

# These lines write each DataFrame to a separate worksheet within the Excel file.
df_pending.to_excel(writer,sheet_name="Summary", index=True)
df_clean.to_excel(writer, sheet_name="All Area", index=False)
df_jkt1.to_excel(writer, sheet_name="JKT 1", index=False)
df_jkt2.to_excel(writer, sheet_name="JKT 2", index=False)
df_jkt3.to_excel(writer, sheet_name="JKT 3", index=False)
df_sby.to_excel(writer, sheet_name="SBY", index=False)
df_bdg.to_excel(writer, sheet_name="BDG", index=False)
df_onl.to_excel(writer, sheet_name="Online", index=False)
df_oth.to_excel(writer, sheet_name="Other", index=False)

writer.close()
   
